In [26]:
# TODO account for main stat of 5-star
# nbd cuz false positives (of good runes) are unlikely:
# they're dragged down by low subs anyways

In [27]:
import numpy as np
import pandas as pd

In [28]:
weights = {'sub_acc': 1, 'sub_res': 1, 'sub_atkp': 1, 'sub_atkf': 0.5, 'sub_defp': 1, 'sub_deff': 0.5, 'sub_hpp': 1, 'sub_hpf': 0.5, 'sub_spd': 2, 'sub_crate': 1, 'sub_cdmg': 1}
# weights_df = pd.DataFrame.from_dict(weights, orient = 'index').T
# weights_df

In [29]:
df = pd.read_csv('runes-data.csv', sep=';')

In [30]:
df = df.drop(columns=['s1_t',	's1_v',	's1_data',
's2_t', 's2_v', 's2_data',
	's3_t',	's3_v',	's3_data',
    	's4_t',	's4_v',	's4_data'])
df = df.replace('-', np.nan)

In [31]:
cols_original = df.columns

In [32]:
# convert inherent stats to eff values, 0-1.
# keep separate because these can't be increased like normal stats

inherent_label_to_sub_label = {'ACC': 'sub_acc', "RES": "sub_res", "ATK%": "sub_atkp", "ATK flat": "sub_atkf",  "DEF%": "sub_defp", "DEF flat": "sub_deff",  "HP%": "sub_hpp", "HP flat": "sub_hpf", "SPD": "sub_spd", "CRate": "sub_crate", "CDmg": "sub_cdmg"}

df['i_t_clean'] = df['i_t'].replace(inherent_label_to_sub_label)

substats_max = {'sub_acc':8, 'sub_res': 8, 'sub_atkp': 8, 'sub_atkf': 20, 'sub_defp': 8, 'sub_deff': 20, 'sub_hpp': 8, 'sub_hpf': 375, 'sub_spd': 6, 'sub_crate': 6, 'sub_cdmg': 7}

df['inh_norm'] = df['i_v'] / (df['i_t_clean'].replace(substats_max)) * (df['i_t_clean'].replace(weights)) 

In [33]:
# convert substats to eff values, 0-5
substats_max = {'sub_acc':8, 'sub_res': 8, 'sub_atkp': 8, 'sub_atkf': 20, 'sub_defp': 8, 'sub_deff': 20, 'sub_hpp': 8, 'sub_hpf': 375, 'sub_spd': 6, 'sub_crate': 6, 'sub_cdmg': 7}

for label in substats_max:
    df[label] = pd.to_numeric(df[label])

for label in substats_max:
    df[label+'_norm'] = df[label]/substats_max[label]*weights[label]


In [34]:
#specify the columns to sum
cols = [str(label+'_norm') for label in substats_max]
cols.append('inh_norm')
#find sum of columns specified 
df['sub_sum_norm'] = df[cols].sum(axis=1)

In [35]:
df['num_powerup_left'] = np.maximum(0, np.subtract(4, np.floor_divide(df['level'], 3)))
df['num_powerup_used'] = np.minimum(4, np.floor_divide(df['level'], 3))


df['num_powerup_incsub'] = df['quality'].map({'Unknown': 0, 'Rare': 2, 'Hero': 3, 'Legend': 4})

df['num_powerup_incsub_left'] = np.maximum(np.subtract(df['num_powerup_incsub'], df['num_powerup_used']) , 0)
df['num_powerup_newsub_left'] = np.subtract(df['num_powerup_left'], df['num_powerup_incsub_left'] )

In [36]:
# From increasing current substats:
# if spd is an increasable substat, then assume all rolls go there
# otherwise assume all rolls go to not-good stats
# TODO this roll could possibly only go to a bad stat (as per norm), not a good one. account for this

df['sub_inc_max_norm'] = (df['num_powerup_incsub_left']*2).where(~df['sub_spd'].isna(), df['num_powerup_incsub_left'])

# assume these are all going to bad stats.
# (even if speed is rollable, probability of going to spd is...low and not worth)
# TODO consider like max vs expected. this straddles the line somewhere
df['sub_new_max_norm'] = df['num_powerup_newsub_left']

In [37]:
# and now sum for the "max" roll eff
cols = ['sub_sum_norm', 'sub_inc_max_norm', 'sub_new_max_norm']

df['tot_max_norm'] = df[cols].sum(axis=1)

In [38]:
pd.options.display.max_columns = None
# df.head(20)
df[(df['level']==0) & (df['quality']=='Rare')].head(5)

,DT_RowId,id,unique_id,monster,originID,monster_n,originName,ancient,efficiency,set,slot,grade,level,m_t,m_v,i_t,i_v,locked,sub_res,sub_cdmg,sub_atkf,sub_acc,sub_atkp,sub_defp,sub_deff,sub_hpp,sub_hpf,sub_spd,sub_crate,quality,max_efficiency,i_t_clean,inh_norm,sub_acc_norm,sub_res_norm,sub_atkp_norm,sub_atkf_norm,sub_defp_norm,sub_deff_norm,sub_hpp_norm,sub_hpf_norm,sub_spd_norm,sub_crate_norm,sub_cdmg_norm,sub_sum_norm,num_powerup_left,num_powerup_used,num_powerup_incsub,num_powerup_incsub_left,num_powerup_newsub_left,sub_inc_max_norm,sub_new_max_norm,tot_max_norm
853,row_2260,2260,43099000578,0,0,Inventory,Inventory,False,39.58,Swift,2,6,0,ATK%,11,NaN,0,0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,Rare,68.15,NaN,NaN,NaN,0.625,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.625000,4,0,2.0,2.0,2.0,2.0,2.0,5.625000
855,row_2279,2279,43114508058,0,0,Inventory,Inventory,False,35.46,Determination,4,5,0,DEF%,8,ATK%,5,0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,Rare,64.03,sub_atkp,0.625,0.625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,2.250000,4,0,2.0,2.0,2.0,4.0,2.0,8.250000
856,row_2282,2282,43114382492,0,0,Inventory,Inventory,False,46.04,Enhance,1,6,0,ATK flat,22,HP%,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,347.0,6.0,NaN,Rare,74.61,sub_hpp,0.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.462667,2.000000,NaN,NaN,3.337667,4,0,2.0,2.0,2.0,4.0,2.0,9.337667
857,row_2284,2284,43114537119,0,0,Inventory,Inventory,False,36.30,Enhance,4,5,0,HP%,8,HP flat,207,0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,5.0,NaN,Rare,64.87,sub_hpf,0.276,NaN,NaN,NaN,NaN,NaN,0.325,NaN,NaN,1.666667,NaN,NaN,2.267667,4,0,2.0,2.0,2.0,4.0,2.0,8.267667
858,row_2285,2285,43113211339,0,0,Inventory,Inventory,False,34.51,Enhance,5,5,0,HP flat,270,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,3.0,NaN,Rare,63.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.200,NaN,NaN,1.000000,NaN,NaN,1.200000,4,0,2.0,2.0,2.0,4.0,2.0,7.200000


In [40]:
cols_export = cols_original
cols_export = cols_export.append(pd.Index(['tot_max_norm']))
df_export = df[cols_export]

In [41]:
# TODO save w timestamp
df_export.to_csv('runes-data-out.csv', sep=';')